# Evaluator Optimizer Agent Workflow
Wir nutzen Best Practices um unsere Agenten zu instanzieren.

In [2]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables and initialize OpenAI client
load_dotenv()

# Azure OpenAI Configuration - following the pattern you showed
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

# Model name should match your Azure deployment name
model = azure_deployment if azure_deployment else "gpt-4o-mini"

if not api_key or not azure_endpoint:
    raise ValueError("Azure OpenAI configuration missing. Please set AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, and AZURE_OPENAI_DEPLOYMENT_NAME in your .env file.")

# Configure client for Azure OpenAI
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

# --- Helper Function for API Calls ---
def call_openai(system_prompt, user_prompt, model=model, temperature=0.0):
    """Simple wrapper for OpenAI API calls."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"
    

# Base Agent Classes
Wir erstellen eine BaseAgent-Klasse. Sie dient als Vorlage und stellt sicher, dass jeder Agent, den wir bauen, eine gemeinsame Struktur hat 
Die execute-Methode ist hier bewusst leer und wirft einen Fehler. Das ist ein Design-Pattern, das uns zwingt, für jeden spezialisierten Agenten, den wir davon ableiten, eine eigene, konkrete execute-Logik zu implementieren.


In [1]:
# agent_classes.py
class BaseAgent:
    """Eine Basisklasse, die die Grundstruktur für alle Agenten definiert."""
    def __init__(self, name: str, persona: str, client):
        self.name = name
        self.persona = persona
        self.client = client

    def execute(self, task: str, context: str = "") -> str:
        """Die Kernmethode, die von spezialisierten Agenten überschrieben wird."""
        raise NotImplementedError("Die 'execute'-Methode muss von einer Subklasse implementiert werden.")

# Spezialisierte Agent Classes
Die ResearchAgent-Klasse erbt von unserer BaseAgent-Klasse. Das bedeutet, sie bekommt automatisch die __init__-Methode. 
Wir implementieren hier nur die execute-Methode. Sie nimmt eine Aufgabe und optionalen Kontext entgegen, verpackt alles in einen System- und einen User-Prompt und ruft die OpenAI-API auf. 


In [ ]:
# agent_classes.py
class Support_Optimizer(BaseAgent):
    """Agent that processes a support ticket and formats the output."""
    def __init__(self):
        name = "Support Agent"
        persona = (
            "Du bist ein erfahrener IT-Support-Mitarbeiter. "
            "Deine einzige Aufgabe ist es, IT-Tickets zu analysieren und eine Empfehlung zu geben. "
            "Deine Antwort muss IMMER ein gültiges JSON-Objekt sein, das KEINEN zusätzlichen Text, keine Markdown-Kommentare, und keine Erklärungen enthält. "
        )
        super().__init__(name, persona, client)

    def execute(self, ticket: str, feedback: str = None) -> str:
        print(f"INFO: {self.name} bearbeitet Ticket: '{ticket}'...")
        system_prompt = self.persona
        user_prompt = f"Analysiere das folgende IT-Ticket und gib eine Empfehlung aus:\n\n'{ticket}'"
        
        # Incorporate feedback from the previous run
        if feedback:
            user_prompt += f"\n\nACHTUNG: Die vorherige Antwort wurde abgelehnt. Grund für die Ablehnung:\n{feedback}\n\nBitte korrigiere deine Antwort basierend auf diesem Feedback. Dein Output MUSS ein gültiges JSON-Objekt im geforderten Format sein."
            
        return call_openai(system_prompt, user_prompt, temperature=0.0)

class Output_Reviewer(BaseAgent):
    """Agent that reviews the output for compliance with the required format using LLM logic."""
    def __init__(self):
        name = "Output Reviewer"
        persona = (
            "Du bist ein strenger Prüfer für die Qualität von JSON-Antworten. "
            "Deine Aufgabe ist es, zu überprüfen, ob ein gegebenes JSON-Objekt alle Kriterien erfüllt. "
            "Deine Antwort muss entweder 'APPROVED' sein, wenn das JSON korrekt ist, oder eine sehr präzise, umsetzbare Begründung für die Ablehnung enthalten."
        )
        super().__init__(name, persona, client)

    def execute(self, output: str) -> str:
        print(f"INFO: {self.name} analysiert den Output...")
        user_prompt = (
            "Ich habe das folgende JSON-Objekt erhalten:\n\n"
            f"```json\n{output}\n```\n\n"
            "Überprüfe strikt, ob dieses JSON-Objekt die folgenden Kriterien erfüllt:\n"
            "1. Es muss ein gültiges JSON-Objekt sein.\n"
            "2. Es darf NUR die folgenden fünf Schlüssel enthalten: 'ticket_nr', 'client', 'company', 'anliegen', 'empfehlung'.\n"
            "3. Alle Werte müssen Strings sein.\n\n"
            "Wenn alle Kriterien erfüllt sind, antworte mit 'APPROVED'. Wenn nicht, erkläre genau, welches Kriterium nicht erfüllt wurde und wie der finale Output ausschauen muss und warum. Du musst immer das volsltändige Feedback geben mit der ganzen Struktur. Deine Antwort muss direkt und ohne zusätzliche Erklärungen sein."
        )
        
        system_prompt = self.persona
        return call_openai(system_prompt, user_prompt, temperature=0.0)


# Konrektes Beispiel
Ein typisches Beispiel aus dem Support-Alltag.

In [44]:
MAX_RETRIES = 5

# The shared user prompt
user_prompt = "Ticket 12432346 Hallo mein Laptop startet nicht... Grüsse Tobias Frei von der Tobias GMBH"

# Der Execution Flow
In unserem Hauptskript importieren wir die Agent-Klassen
Dann instanziieren wir sie: Wir erstellen ein konkretes Objekt namens ticket_agent eval_agent.
Dann loopen wir über die Schleife, bis die Max tries erreicht werden, oder der Evaluator das Wort "approved" ausspuckt.

In [41]:
def main():
    ticket_agent = Support_Optimizer()
    eval_agent = Output_Reviewer()

    ticket_output = ""
    feedback = None

    for attempt in range(MAX_RETRIES):
        print(f"--- Attempt #{attempt} ---")
        ticket_output = ticket_agent.execute(user_prompt, feedback)
        print(f"Agent Output:\n{ticket_output}\n")
        
        evaluation = eval_agent.execute(ticket_output)

        print(f"\n🧾 Evaluation Result:\n{evaluation}\n")

        if evaluation.lower().startswith("approved"):
            print("\n✅ Final Approved Investment Summary:\n")
            print(ticket_output)
            break
        else:
            feedback = evaluation
            
    else:
        print("\n❌ Failed to meet compliance after max retries.")
        print("Last version of the report:")
        print(ticket_output)

if __name__ == "__main__":
    main()



--- Attempt #0 ---
INFO: Support Agent bearbeitet Ticket: 'Ticket 12432346 Hallo mein Laptop startet nicht... Grüsse Tobias Frei von der Tobias GMBH'...
Agent Output:
{
  "ticket_id": 12432346,
  "issue_summary": "Laptop startet nicht",
  "recommended_action": "Überprüfen Sie, ob der Laptop an eine Stromquelle angeschlossen ist und der Akku geladen ist. Versuchen Sie einen Hard-Reset durch langes Drücken des Einschaltknopfs. Falls das Problem weiterhin besteht, sollte der Laptop von der IT-Abteilung überprüft werden, um Hardwaredefekte auszuschließen.",
  "priority": "hoch",
  "assigned_to": "Hardware-Support"
}

INFO: Output Reviewer analysiert den Output...

🧾 Evaluation Result:
Das JSON-Objekt enthält nicht die geforderten Schlüssel. Es enthält die Schlüssel 'ticket_id', 'issue_summary', 'recommended_action', 'priority' und 'assigned_to', aber es dürfen nur die Schlüssel 'ticket_nr', 'client', 'company', 'anliegen' und 'empfehlung' enthalten sein. Außerdem sind nicht alle Werte Stri

# In production

In [45]:
# --- Function to process a ticket ---
def process_ticket(ticket: str, max_retries: int = 5) -> str:
    """
    Processes a support ticket using an iterative agent-based approach.
    
    Args:
        ticket (str): The support ticket text.
        max_retries (int): The maximum number of attempts before failing.
        
    Returns:
        str: The final, approved JSON output or an error message.
    """
    ticket_agent = Support_Optimizer()
    eval_agent = Output_Reviewer()

    ticket_output = ""
    feedback = None

    for attempt in range(max_retries):
        print(f"--- Attempt #{attempt} ---")
        ticket_output = ticket_agent.execute(ticket, feedback)
        print(f"Agent Output:\n{ticket_output}\n")
        
        evaluation = eval_agent.execute(ticket_output)

        print(f"\n🧾 Evaluation Result:\n{evaluation}\n")

        if evaluation.lower().startswith("approved"):
            print(f"\n✅ Erfolgreich nach {attempt + 1} Versuchen abgeschlossen.")
            return ticket_output
        else:
            feedback = evaluation
            
    print(f"\n❌ Fehler: Konnte die Anforderungen nach {max_retries} Versuchen nicht erfüllen.")
    print("Letzter generierter Bericht:")
    return ticket_output

# --- Example of how to call the new function ---
if __name__ == "__main__":
    # Example 1: a successful ticket
    user_prompt_1 = "Ticket 12432346 Hallo mein Laptop startet nicht... Grüsse Tobias Frei von der Tobias GMBH"
    final_result_1 = process_ticket(user_prompt_1)
    
    print("\n\n=== Final Approved Output for Example 1 ===")
    print(final_result_1)

    print("\n" + "="*50 + "\n")

    # Example 2: a more complex ticket that might require more attempts
    user_prompt_2 = "Support-Anfrage 99876543: Mein Drucker druckt keine Farben mehr. Er zeigt einen Fehlercode 'E-45' an. Absender: Franziska Müller, Müller & Söhne GmbH."
    final_result_2 = process_ticket(user_prompt_2)
    
    print("\n\n=== Final Approved Output for Example 2 ===")
    print(final_result_2)

--- Attempt #0 ---
INFO: Support Agent bearbeitet Ticket: 'Ticket 12432346 Hallo mein Laptop startet nicht... Grüsse Tobias Frei von der Tobias GMBH'...
Agent Output:
{
  "ticket_id": 12432346,
  "issue_summary": "Laptop startet nicht",
  "recommendation": "Überprüfen Sie, ob der Laptop an eine Stromquelle angeschlossen ist. Versuchen Sie einen Hard-Reset durch langes Drücken des Power-Buttons. Wenn das Problem weiterhin besteht, bringen Sie das Gerät zur weiteren Diagnose in die IT-Abteilung."
}

INFO: Output Reviewer analysiert den Output...

🧾 Evaluation Result:
Das JSON-Objekt ist zwar gültig, aber es erfüllt nicht die Kriterien 2 und 3:

- Kriterium 2: Das Objekt enthält die Schlüssel "ticket_id", "issue_summary" und "recommendation", die nicht erlaubt sind. Erlaubt sind nur die Schlüssel: "ticket_nr", "client", "company", "anliegen", "empfehlung".
- Kriterium 3: Der Wert von "ticket_id" ist eine Zahl (12432346), kein String.

Das finale JSON-Objekt muss also genau diese fünf Schl